In [1]:
import altair as alt
import numpy as np
import pandas as pd

In [2]:
fl = pd.read_csv("/Users/emma/Downloads/FL_WA_DEC10_subsets_no_dups/FL_DEC10_subset.csv")
wa = pd.read_csv("/Users/emma/Downloads/FL_WA_DEC10_subsets_no_dups/WA_DEC10_subset.csv")


In [3]:
FL_DE = pd.read_csv("/Users/emma/Downloads/DE_VA_dupsremoved_DEC11/DE subsets_regen_DEC11.csv")
FL = pd.concat([fl, FL_DE])

In [4]:
FL = FL[['BUYER_STATE_x', 'BUYER_COUNTY_x', 'TRANSACTION_YEAR',
       'TRANSACTION_DATE_DT', 'Opioids_Shipment_IN_GM', 'fips',
       'opioid_shipment_population_ratio', 'year', 'population', 'deaths']]

In [40]:
FL.to_csv("fl_summary.csv")

In [41]:
WA.to_csv("wa_summary.csv")

In [5]:
WA_VA = pd.read_csv("/Users/emma/Downloads/DE_VA_dupsremoved_DEC11/VA subsets_regen_DEC11.csv")
WA_VA = WA_VA[['BUYER_STATE_x', 'BUYER_COUNTY_x', 'TRANSACTION_YEAR',
       'TRANSACTION_DATE_DT', 'Opioids_Shipment_IN_GM', 'fips',
       'opioid_shipment_population_ratio', 'year', 'population', 'deaths']]

In [6]:
WA = pd.concat([wa, WA_VA])

In [7]:
def data_opioids(df, year, state):
    """make dataset ready to run diff-diff and pre-post for opioids"""
    df["TRANSACTION_YEAR"] = df["TRANSACTION_YEAR"].astype("int")
    df = (df.groupby(['BUYER_STATE_x','BUYER_COUNTY_x','TRANSACTION_YEAR','population','deaths'],as_index=False)['opioid_shipment_population_ratio'].sum())
    df["policy"] = 0
    df.loc[df["TRANSACTION_YEAR"] > year, "policy"] = 1
    df["state"] = 0
    df.loc[df["BUYER_STATE_x"] == state, "state"] = 1
    return df



In [8]:
def data_death(df, year, state):
    df["TRANSACTION_YEAR"] = df["TRANSACTION_YEAR"].astype("int")
    df["Deaths"] = df["Deaths"].astype("int")
    df["death_rate"]= df["Deaths"]/df["population"]
    df["death_rate"]*= 100000
    df = df.groupby(['BUYER_STATE_x','BUYER_COUNTY_x','TRANSACTION_YEAR'],as_index=False)['death_rate'].sum()
    df["policy"] = 0
    df.loc[df["TRANSACTION_YEAR"] > year, "policy"] = 1
    df["state"] = 0
    df.loc[df["BUYER_STATE_x"] == state, "state"] = 1
    
    return df

In [9]:
FL = data_opioids(FL, 2010, "FL")

In [10]:
FL_pre = pd.read_csv("/Users/emma/pds-2022-leep/20_intermediate_files/fl_pre.csv")
FL_post = pd.read_csv("/Users/emma/pds-2022-leep/20_intermediate_files/fl_post.csv")
FL_new = pd.concat([FL_pre, FL_post])
FL_new = FL_new.rename(columns = {"year":"TRANSACTION_YEAR", "state": "BUYER_STATE_x", "County":"BUYER_COUNTY_x", "deaths": "Deaths"})

In [11]:
FL_death = data_death(FL_new, 2010, "FL")

In [12]:
WA_full = pd.read_csv("/Users/emma/pds-2022-leep/20_intermediate_files/WA_full.csv")
WA_new = WA_full.rename(columns = {"year":"TRANSACTION_YEAR", "state": "BUYER_STATE_x", "County":"BUYER_COUNTY_x", "deaths": "Deaths"})

In [14]:
WA_death = data_death(WA_new, 2012, "WA")

In [15]:
WA = data_opioids(WA, 2012, "WA")

In [16]:
TX_pre = pd.read_csv("/Users/emma/pds-2022-leep/20_intermediate_files/tx_pre.csv")
TX_post = pd.read_csv("/Users/emma/pds-2022-leep/20_intermediate_files/tx_post.csv")
TX_new = pd.concat([TX_pre, TX_post])
TX_new = TX_new.rename(columns = {"year":"TRANSACTION_YEAR", "state": "BUYER_STATE_x", "County":"BUYER_COUNTY_x", "deaths": "Deaths"})

In [17]:
TX_death = data_death(TX_new, 2007, "TX")

In [18]:
def pre_post(data, yvar, xvar, year, analysis, target, alpha=0.05):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    data1 = data.loc[(data["policy"]==0) & (data["state"] ==1), :]
    x = data1.loc[pd.notnull(data1[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions1 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before = smf.ols(f"{yvar} ~ {xvar}", data=data1).fit()
    model_predict = model_before.get_prediction(predictions1[xvar])
    predictions1[yvar] = model_predict.summary_frame()["mean"]
    predictions1[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg1 = alt.Chart(predictions1).mark_line().encode(x=alt.X(xvar, axis=alt.Axis(format='O', title = "Year")), y = alt.Y(yvar, scale= alt.Scale(zero=False)))
    ci1 = (
        alt.Chart(predictions1)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", axis=alt.Axis(title= target)),
            y2="ci_high",
        )
    )
    
    data2 = data.loc[(data["policy"]==1) & (data["state"]==1),:]
    x = data2.loc[pd.notnull(data2[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions2 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after = smf.ols(f"{yvar} ~ {xvar}", data=data2).fit()
    model_predict_after = model_after.get_prediction(predictions2[xvar])
    predictions2[yvar] = model_predict_after.summary_frame()["mean"]
    predictions2[["ci_low", "ci_high"]] = model_predict_after.conf_int(alpha=alpha)

    # Build chart
    reg2 = alt.Chart(predictions2).mark_line().encode(x=alt.X(xvar, axis=alt.Axis(format='O', title = "Year")), y= alt.Y(yvar, scale= alt.Scale(zero=False)))
    ci2 = (
        alt.Chart(predictions2)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", axis=alt.Axis(title= target)),
            y2="ci_high",
        )
    )

    overlay = pd.DataFrame({'x': [year]})
    vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')

    chart = alt.layer(ci1, ci2,reg1, reg2, vline).properties(title = analysis)
    
    return predictions1, chart

In [34]:
def diff_diff(data, yvar, xvar, year, analysis, target, state, alpha=0.05):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    data1 = data.loc[(data["policy"]==0) & (data["state"] == 0),:]
    x = data1.loc[pd.notnull(data1[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions1 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before = smf.ols(f"{yvar} ~ {xvar}", data=data1).fit()
    model_predict = model_before.get_prediction(predictions1[xvar])
    predictions1[yvar] = model_predict.summary_frame()["mean"]
    predictions1[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg1 = alt.Chart(predictions1).mark_line(color="green").encode(x = xvar, y = alt.Y(yvar, scale= alt.Scale(zero=False)))
    ci1 = (
        alt.Chart(predictions1)
        .mark_errorband(color="green")
        .encode(
            x=xvar,
            y=alt.Y("ci_low", axis=alt.Axis(title= target)),
            y2="ci_high",
        )
    )

    data11 = data.loc[(data["policy"]==0) & (data["state"] == 1),:]
    x = data11.loc[pd.notnull(data11[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions11 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before1 = smf.ols(f"{yvar} ~ {xvar}", data=data11).fit()
    model_predict1 = model_before1.get_prediction(predictions11[xvar])
    predictions11[yvar] = model_predict1.summary_frame()["mean"]
    predictions11[["ci_low", "ci_high"]] = model_predict1.conf_int(alpha=alpha)

    # Build chart
    reg11 = alt.Chart(predictions11).mark_line().encode(x=xvar, y = alt.Y(yvar, scale= alt.Scale(zero=False)))
    ci11 = (
        alt.Chart(predictions11)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", axis=alt.Axis(title= target)),
            y2="ci_high",
        )
    )
    
    data2 = data.loc[(data["policy"]==1) & (data["state"] == 0),:]
    x = data2.loc[pd.notnull(data2[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions2 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after = smf.ols(f"{yvar} ~ {xvar}", data=data2).fit()
    model_predict_after = model_after.get_prediction(predictions2[xvar])
    predictions2[yvar] = model_predict_after.summary_frame()["mean"]
    predictions2[["ci_low", "ci_high"]] = model_predict_after.conf_int(alpha=alpha)

    # Build chart
    reg2 = alt.Chart(predictions2).mark_line(color="green").encode(x=xvar, y = alt.Y(yvar, scale= alt.Scale(zero=False)))
    ci2 = (
        alt.Chart(predictions2)
        .mark_errorband(color="green")
        .encode(
            x=xvar,
            y=alt.Y("ci_low", axis=alt.Axis(title= target)),
            y2="ci_high"
        )
    )

    data21 = data.loc[(data["policy"]==1) & (data["state"] == 1),:]
    x = data21.loc[pd.notnull(data21[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions21 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after1 = smf.ols(f"{yvar} ~ {xvar}", data=data21).fit()
    model_predict_after1 = model_after1.get_prediction(predictions21[xvar])
    predictions21[yvar] = model_predict_after1.summary_frame()["mean"]
    predictions21[["ci_low", "ci_high"]] = model_predict_after1.conf_int(alpha=alpha)

    # Build chart
    reg21 = alt.Chart(predictions21).mark_line().encode(x=alt.X(xvar, axis=alt.Axis(format='O', title = "Year")), y=yvar)
    ci21 = (
        alt.Chart(predictions21)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", axis=alt.Axis(title= target)),
            y2="ci_high",
        )
    )
    label1 = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=12,
    fontWeight=500,
    color='green').encode(
    x=alt.value(275),  
    y=alt.value(260),  
    text=alt.value(["―Constant States"]))
    label2 = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=12,
    fontWeight=500, color = "#1f77b4").encode(
    x=alt.value(220),  
    y=alt.value(260),  
    text=alt.value([f"―{state}"]))

    # text = alt.Chart().mark_text(
    # align="left",
    # baseline="bottom",
    # lineBreak = ".",
    # fontSize=12,
    # fontWeight=400,
    # ).encode(
    # x=alt.value(0),  
    # y=alt.value(350),  
    # text=alt.value([f"Note: In the graph analyzing {state} regarding its {target}.", "\n constant states to compare include.", f"{constant_states}"]))

    overlay = pd.DataFrame({'x': [year]})
    vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=2).encode(x='x:Q') 
   
    chart = alt.layer(ci1, ci2, ci11, ci21, reg1, reg11, reg2, reg21, label1, label2, vline).properties(title = analysis)
    
    return chart

## Florida

> pre-post opioids

In [20]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
analysis = "Figure 1. Pre-Post Analysis For Florida"
target = "Opioids Shipment Per Capita (in grams)"
fit, chart = pre_post(FL, yvar, xvar, 2010, analysis, target, alpha=0.05)
chart

alt.LayerChart(...)

> diff-diff opioids

In [35]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
analysis = "Figure 2. Difference-Difference Analysis For Florida"
target = "Opioids Shipment Per Capita (in grams)"
state = "Florida"
chart = diff_diff(FL, yvar, xvar, 2010, analysis, target, state)
chart

alt.LayerChart(...)

> pre-post death rate

In [22]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
analysis = "Figure 3. Pre-Post Analysis For Florida"
target = "Overdose Death Rate (per 100,000 capita)"
fit, chart = pre_post(FL_death, yvar, xvar, 2010, analysis, target, alpha=0.05)
chart

alt.LayerChart(...)

> diff-diff for death rate

In [23]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
analysis = "Figure 4. Difference-Diference Analysis For Florida"
target = "Overdose Death Rate (per 100,000 capita)"
constant_states = "Maine,West Virginia,Vermont,Delaware,Hawaii,Montana,Pennsylvania,New Hampshire,South Carolina, New Mexico"
chart = diff_diff(FL_death, yvar, xvar, 2010, analysis, target, state, alpha=0.05)
chart

alt.LayerChart(...)

## Washington

> pre-post opioids

In [24]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
analysis = "Figure 5. Pre-Post Analysis For Washington"
target = "Opioids Shipment per Capita (in grams)"
fit, chart = pre_post(WA, yvar, xvar, 2012, analysis, target, alpha=0.05)
chart

alt.LayerChart(...)

> diff-diff opioids

In [25]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
analysis = "Figure 6. Difference-Difference Analysis For Washington"
target = "Opioids Shipment Per Capita (in grams)"
state = "Washington"
constant_states = "Louisiana, Maryland, Oklahoma, Washington, Indiana, Idaho, Minnesota, Nebraska, Nevada, Virginia"
chart = diff_diff(WA, yvar, xvar, 2012, analysis, target, state)
chart

alt.LayerChart(...)

> pre-post for death rate

In [26]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
analysis = "Figure 7. Pre-Post Analysis For Washington"
target = "Overdose Death Rate (per 100,000 capita)"
fit, chart = pre_post(WA_death, yvar, xvar, 2012, analysis, target, alpha=0.05)
chart

alt.LayerChart(...)

> diff-diff for death rate

In [28]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
analysis = "Figure 8. Difference-Difference Analysis for Washington"
target = "Overdose Death Rate (per 100,000 capita)"
state = "Washington"
chart = diff_diff(WA_death, yvar, xvar, 2012, analysis, target, state)
chart

alt.LayerChart(...)

## Texas

In [29]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
analysis = "Figure 9. Pre-Post Analysis For Texas"
target = "Overdose Death Rate (per 100,000 capita)"
fit, chart = pre_post(TX_death, yvar, xvar, 2007, analysis, target, alpha=0.05)
chart

alt.LayerChart(...)

In [30]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
analysis = "Figure 10. Difference-Difference Analysis for Texas"
target = "Overdose Death Rate (per 100,000 capita)"
state = "Texas"
chart = diff_diff(TX_death, yvar, xvar, 2007, analysis, target, state)
chart

alt.LayerChart(...)

In [37]:
FL_death.BUYER_STATE_x.unique() == FL.BUYER_STATE_x.unique()

array([ True,  True,  True,  True,  True,  True,  True,  True])

In [38]:
WA_death.BUYER_STATE_x.unique() == WA.BUYER_STATE_x.unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [47]:
def estimate_diff(df, target):
    """doing calculations for the effect of policy change (based on diff-diff analysis)"""
    effect = []
    for state in [0, 1]:
        df_pre = df.loc[(df["policy"]==0) & (df["state"] == state),:]
        pre = df_pre[target].mean()
        print(f"the prepolicy for {target} is {pre}")
        df_post = df.loc[(df["policy"]==1) & (df["state"] == state),:]
        post = df_post[target].mean()
        print(f"the postpolicy for {target} is {post}")
        effect.append(post - pre)
        print(effect)
    return effect[1] - effect[0]

In [48]:
print(estimate_diff(WA, "opioid_shipment_population_ratio"))


the prepolicy for opioid_shipment_population_ratio is 0.19634674408517794
the postpolicy for opioid_shipment_population_ratio is 0.20468503083085407
[0.008338286745676132]
the prepolicy for opioid_shipment_population_ratio is 0.1796266148901012
the postpolicy for opioid_shipment_population_ratio is 0.16503857975552017
[0.008338286745676132, -0.01458803513458104]
-0.022926321880257172


In [49]:
print(estimate_diff(FL, "opioid_shipment_population_ratio"))

the prepolicy for opioid_shipment_population_ratio is 0.21989815092880022
the postpolicy for opioid_shipment_population_ratio is 0.2890790052340006
[0.06918085430520038]
the prepolicy for opioid_shipment_population_ratio is 0.3516838842444628
the postpolicy for opioid_shipment_population_ratio is 0.2915070450591304
[0.06918085430520038, -0.0601768391853324]
-0.12935769349053278


In [50]:
print(estimate_diff(FL_death, "death_rate"))

the prepolicy for death_rate is 14.85639044126355
the postpolicy for death_rate is 19.87993304934507
[5.02354260808152]
the prepolicy for death_rate is 16.299365596888922
the postpolicy for death_rate is 14.694261381586818
[5.02354260808152, -1.605104215302104]
-6.628646823383624


In [51]:
print(estimate_diff(WA_death, "death_rate"))

the prepolicy for death_rate is 12.485411539183557
the postpolicy for death_rate is 15.184035483039539
[2.6986239438559814]
the prepolicy for death_rate is 12.784897581200894
the postpolicy for death_rate is 12.241633369129389
[2.6986239438559814, -0.5432642120715059]
-3.2418881559274872


In [52]:


print(estimate_diff(TX_death, "death_rate"))

the prepolicy for death_rate is 11.567675133495785
the postpolicy for death_rate is 13.970450576373361
[2.4027754428775765]
the prepolicy for death_rate is 10.066350717304863
the postpolicy for death_rate is 10.999959859157265
[2.4027754428775765, 0.9336091418524024]
-1.469166301025174
